# Portfolio Optimization in Python 101 - PyPortfolioOpt edition

## Setup

In [1]:
import requests
import pandas as pd
import numpy as np
import quantstats as qs
from pypfopt.expected_returns import mean_historical_return

# api key
from api_keys import FMP_API_KEY

ModuleNotFoundError: No module named 'pypfopt'

## Downloading data

In [ ]:
FAANG_TICKERS = ["META", "AAPL", "AMZN", "NFLX", "GOOGL"]
START_DATE = "2023-01-01"

In [ ]:
def get_adj_close_price(symbol, start_date):
    hist_price_url = f"https://financialmodelingprep.com/api/v3/historical-price-full/{symbol}?from={start_date}&apikey={FMP_API_KEY}"
    r_json = requests.get(hist_price_url).json()
    df = pd.DataFrame(r_json["historical"]).set_index("date").sort_index()
    df.index = pd.to_datetime(df.index)
    return df[["adjClose"]].rename(columns={"adjClose": symbol})

In [ ]:
price_df_list = []
for ticker in FAANG_TICKERS:
    price_df_list.append(get_adj_close_price(ticker, START_DATE))
prices_df = price_df_list[0].join(price_df_list[1:])
prices_df = prices_df[:"2024-05-02"]
prices_df

In [ ]:
returns_df = prices_df.pct_change().dropna()
returns_df

In [ ]:
returns_df = mean_historical_return(prices_df, compounding=False)

## Portfolio Optimization

In [ ]:
# Calculate the annualized expected returns and the covariance matrix
avg_returns = returns_df.mean() * 252
cov_mat = returns_df.cov() * 252

# Define the function to find the portfolio volatility using the weights and the covariance matrix
def get_portfolio_volatility(weights, cov_mat):
    return np.sqrt(np.dot(weights.T, np.dot(cov_mat, weights)))

# Define the number of assets
n_assets = len(avg_returns)

# Define the bounds - the weights can be between 0 and 1
bounds = tuple((0, 1) for asset in range(n_assets))

# Define the initial guess - the equally weighted portfolio
initial_guess = n_assets * [1.0 / n_assets]

# Define the constraint - all weights must add up to 1
constr = {"type": "eq", "fun": lambda x: np.sum(x) - 1}

# Find the minimum volatility portfolio
min_vol_portf = sco.minimize(
    get_portfolio_volatility,
    x0=initial_guess,
    args=cov_mat,
    method="SLSQP",
    constraints=constr,
    bounds=bounds,
)

# Store the portfolio weights
min_vol_portf_weights = pd.Series(min_vol_portf.x, index=avg_returns.index).round(2)
min_vol_portf_weights

In [ ]:
min_vol_portf.fun

In [ ]:
def get_neg_sharpe_ratio(weights, avg_rtns, cov_mat, rf_rate):
    portf_returns = np.sum(avg_rtns * weights)
    portf_volatility = np.sqrt(np.dot(weights.T, np.dot(cov_mat, weights)))
    portf_sharpe_ratio = (portf_returns - rf_rate) / portf_volatility
    return -portf_sharpe_ratio

In [ ]:
RF_RATE = 0
args = (avg_returns, cov_mat, RF_RATE)

max_sharpe_portf = sco.minimize(
    get_neg_sharpe_ratio,
    x0=initial_guess,
    args=args,
    method="SLSQP",
    bounds=bounds,
    constraints=constr,
)

# Store the portfolio weights
max_sharpe_portf_weights = pd.Series(max_sharpe_portf.x, index=avg_returns.index).round(2)
max_sharpe_portf_weights

In [ ]:
max_sharpe_portf.fun

## Evaluating the Portfolios

In [ ]:
min_vol_portf_returns = returns_df.dot(min_vol_portf_weights)
qs.reports.metrics(min_vol_portf_returns, benchmark="SPY", mode="basic", rf=0)

In [ ]:
max_sharpe_portf_returns = returns_df.dot(max_sharpe_portf_weights)
qs.reports.metrics(max_sharpe_portf_returns, benchmark="SPY", mode="basic", rf=0)

In [ ]:
ef = EfficientFrontier(expected_returns, cov_matrix)  # setup
ef.add_objective(objective_functions.L2_reg)  # add a secondary objective
ef.min_volatility()  # find the portfolio that minimises volatility and L2_reg
